In [8]:
import pandas as pd
import glob
import requests
from bs4 import BeautifulSoup
import os, shutil
from datetime import datetime
from time import sleep
import urllib.request
from zipfile import ZipFile
# import wget

class Cvm():

    def __init__(self, url):

        self.url = url
        self.html = ''
        self.soup = ''
        self.pages = []
        self.files = []
        self.file_ = []

    def get_parser(self):

        self.html = requests.get(self.url)

        self.soup = BeautifulSoup(self.html.text, 'html.parser')

    def get_status(self):

        print(self.html.status_code)

    def get_files(self, lista):

        ''' Funcao que percorre todas as paginas selecionadas e salva os links
        para a funcao de download '''
        # print('lista de logs:')
        for file in lista:

            self.html = requests.get(self.url + file)

            self.soup = BeautifulSoup(self.html.text, 'html.parser')

            # Buscando a ultima data de atualizacao do arquivo
            date = self.soup.find('table',
                             {'class': 'table table-striped table-bordered table-condensed'})\
                             .find('span',
                                   {'class': 'automatic-local-datetime'})\
                                   .getText()\
                                   .strip()

            last_update = f'{self.url + file} - {date}'
            # print(last_update)

            ''' Bloco que tenta ler o arquivo de log ou cria um novo, caso nao encontre
                Se existir um arquivo com mesmo nome e a mesma data e hora de atualizacao
                no arquivo de log, o arquivo nao sera adicionado a lista para download '''
            try:
                with open('ultima_atualizacao.txt', 'r') as f:
                    content = f.read()
                    if last_update in content:
                        pass
                    else:
                        for link in self.soup.find('section', {'id': 'dataset-resources'}).find_all('a'):
                            self.file_.append(link.get('href').split('\n'))
                        with open('ultima_atualizacao.txt', 'w') as f:
                            f.write(content)
                            f.write(last_update)
                            f.write('\n')
            except:
                with open('ultima_atualizacao.txt', 'w') as f:
                    f.write(last_update)
                    f.write('\n')
                    for link in self.soup.find('section', {'id': 'dataset-resources'}).find_all('a'):
                        self.file_.append(link.get('href').split('\n'))


    def download(self, output = './', date_ignore=None, ano='2022'):
        
        ''' Funcao que faz o download de todos os arquivos mapeados no diretorio
        selecionado '''

        print('Arquivos baixados:')
        for i in self.file_:
            if ('http' in i[0]) & ('2018' not in i[0]) & ('2019' not in i[0]) \
                & ('/META/' not in i[0]) & (i[0][i[0].rfind(ano):i[0].rfind(ano)+6] not in date_ignore):

                ''' Olha no diretorio de armazenamento dos arquivos,
                    caso haja uma atualizacao e o arquivo esteja na lista,
                    sera removido o arquivo anterior e salvo a nova versao '''
                file_name = i[0]
                if str(file_name).split('/')[-1] in os.listdir(output):
                    # print(file_name)
                    try:
                        os.remove(f'{output}/{file_name.split("/")[-1]}')
                        sleep(2)
                        # wget.download(file_name, out = output)
                        urllib.request.urlretrieve(file_name, output + str(file_name).split('/')[-1])
                        print(file_name)
                    except:
                        os.remove(f'{output}{str(file_name).split("/")[-1]}')
                        sleep(2)
                        # wget.download(file_name, out = output)
                        urllib.request.urlretrieve(file_name, output + str(file_name).split('/')[-1])
                        print(file_name)
                else:
                    # wget.download(file_name, out = output)
                    urllib.request.urlretrieve(file_name, output + str(file_name).split('/')[-1])
                    print(file_name)

In [9]:
# Caminho da pagina principal
url = 'https://dados.cvm.gov.br/dataset'

# Caminho das extensoes
endpoint = ['/fi-cad',
         '/cia_aberta-cad',
         '/fii-doc-inf_trimestral',
         '/cia_aberta-doc-itr',
         '/fidc-doc-inf_mensal']

# Datas a serem ignoradas
ignore = ['202201', '202202', '202203', '202204', '202205', '202206',
          '202207', '202208', '202209']

# Chamada das funcoes de download
caminho_download = '../dados/'

cvm = Cvm(url)
cvm.get_parser()
cvm.get_files(endpoint)
cvm.download(caminho_download, date_ignore=ignore)

Arquivos baixados:
https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv
https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi_hist.zip
https://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/cad_cia_aberta.csv
https://dados.cvm.gov.br/dados/FII/DOC/INF_TRIMESTRAL/DADOS/inf_trimestral_fii_2020.zip
https://dados.cvm.gov.br/dados/FII/DOC/INF_TRIMESTRAL/DADOS/inf_trimestral_fii_2021.zip
https://dados.cvm.gov.br/dados/FII/DOC/INF_TRIMESTRAL/DADOS/inf_trimestral_fii_2022.zip
https://dados.cvm.gov.br/dados/FII/DOC/INF_TRIMESTRAL/DADOS/inf_trimestral_fii_2023.zip
https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_2020.zip
https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_2021.zip
https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_2022.zip
https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_2023.zip
https://dados.cvm.gov.br/dados/FIDC/DOC/INF_MENSAL/DADOS/inf_mensal_fidc_202210.zip
https://dados.cvm.gov.br/dados/FIDC/DOC